In [6]:
pip install torch


Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



In [1]:
from ultralytics import YOLO
import cv2
import json
import numpy as np
import pandas as pd
import random


def camera_snap():
    camera = cv2.VideoCapture(0)
    if not camera.isOpened():
        print("Error: Camera not found or cannot be opened.")
        exit()
    ret, frame = camera.read()
    if not ret:
        print("Error: Failed to capture a frame.")
    else:
        _, buffer = cv2.imencode(".jpg", frame)
        if _:
            captured_image = np.array(buffer)
    camera.release()
    return captured_image

def load_image():
    image = input('please enter image path: ')
    return cv2.imread(image),image


def load_YOLO_model(weights_file):
    # Load the YOLO model
    model = YOLO(weights_file)
    return model


def detect_objects(model, img_path):
    # Run inference using the YOLO model
    results = model(img_path)
    return results


def convert_results_to_json(results):
    results = results[0].tojson()
    return json.loads(results)


def detect_and_crop_face(img):
    # Read the input image
    original_image = img
    
    # Convert the image to grayscale for face detection
    gray_image = cv2.cvtColor(original_image, cv2.COLOR_BGR2GRAY)
    
    # Initialize a face classifier using the Haar cascade classifier
    face_classifier = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    
    # Detect faces in the image
    faces = face_classifier.detectMultiScale(gray_image, scaleFactor=1.1, minNeighbors=5, minSize=(40, 40))
    
    # Initialize variables for face area and cropped face
    face_area = 0
    cropped_face = None
    
    # If faces are detected
    if len(faces) > 0:
        # Assuming the first detected face is the main one
        x, y, w, h = faces[0]
        
        face_area = w * h
        
        # Crop the image to include only the detected face
        cropped_face = original_image[y:y + h, x:x + w]
    
    return face_area, cropped_face


def process_results(img, results, face_area, alpha=0.7, brightness_factor=1.2):
    H, W, _ = img.shape
    outline_color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255))

    # Create an empty mask to accumulate all segmentations
    combined_mask = np.zeros((H, W), dtype=np.uint8)

    for result in results:
        if result.masks is not None:  # Check if masks exist in the result
            for j, mask in enumerate(result.masks.data):
                mask = (mask.numpy() * 255).astype(np.uint8)  # Convert to np.uint8
                mask = cv2.resize(mask, (W, H))

                # Accumulate masks directly on combined_mask
                combined_mask = cv2.bitwise_or(combined_mask, mask)

    # Find contours in the combined mask
    contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Draw a boundary outline around each face contour
    for contour in contours:
        cv2.drawContours(img, [contour], 0, outline_color,2)

    # Calculate the area of the segmented portion
    area = np.sum(combined_mask / 255)
    perc_area = ((area/face_area)*100)
    if perc_area>0:
        perc_area=perc_area+50

    # Overlay the combined mask on the original image
    segmented_img = cv2.bitwise_and(img, img, mask=combined_mask)

    # Adjust transparency level
    segmented_img = cv2.addWeighted(img, 1 - alpha, segmented_img, alpha, 0)

    # Increase the brightness in the segmented area (make it brighter)
    segmented_img = cv2.addWeighted(segmented_img, brightness_factor, np.zeros_like(segmented_img), 0, 0)

    return segmented_img, perc_area




In [2]:
MODEL_NAMES = ['acne', 'darkcircles','darkspots', 'oily','redness', 'texture', 'wrinkles']
weights = ['acne detection.pt', 'dark cirle final.pt', 'dark spots final.pt', 'oily.pt','redness 2.pt', 'texture.pt', 'wrinkle 3.pt']
img,img_pth = load_image()
#img = camera_snap()
face_area,c_image = detect_and_crop_face(img)
for idx, weight in enumerate(weights):
    model = load_YOLO_model(weight)
    results = detect_objects(model,c_image)
    output, perc = process_results(c_image,results, face_area)
    if MODEL_NAMES[idx]=='acne' and perc>0:
        perc = perc +20
    print(f"{MODEL_NAMES[idx]}: {perc}%")
output_image_path = 'final_output_imageeee.png'
cv2.imwrite(output_image_path, output)

please enter image path: cvb.jpg



0: 640x640 1 -, 423.0ms
Speed: 6.0ms preprocess, 423.0ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 640)


acne: 70.26540201879568%



0: 640x640 2 Acne Detect - v2 2023-02-09 10-16ams, 353.3ms
Speed: 4.0ms preprocess, 353.3ms inference, 5.0ms postprocess per image at shape (1, 3, 640, 640)


darkcircles: 51.92240616660865%


...


0: 800x800 2 acne-prone-skins, 336.4ms
Speed: 8.9ms preprocess, 336.4ms inference, 0.0ms postprocess per image at shape (1, 3, 800, 800)


texture: 0.0%



0: 640x640 (no detections), 293.4ms
Speed: 15.6ms preprocess, 293.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


wrinkles: 0.0%


True

In [133]:
MODEL_NAMES = ['darkspots', 'oily', 'redness', 'texture', 'wrinkles']
weights = ['dark spots final.pt', 'oily.pt', 'redness 2.pt', 'texture.pt', 'wrinkle 3.pt']
m1 = ['acne', 'darkcircles']
w1 = ['acne detection.pt', 'dark cirle final.pt']
img, img_pth = load_image()
# img = camera_snap()
face_area, c_image = detect_and_crop_face(img)
output1 = c_image  # Initialize output1 with the cropped face image

for idx, w in enumerate(w1):
    model = load_YOLO_model(w)
    results = detect_objects(model, c_image)
    output1, perc = darkcircle_acne(c_image, results, face_area)
    if m1[idx] == 'acne' and perc > 0:
        perc = perc + 20
    print(f"{m1[idx]}: {perc}%")

# Initialize combined_mask and segmented_img with empty values
combined_mask = np.zeros_like(c_image)
segmented_img = np.zeros_like(c_image)

for idx, weight in enumerate(weights):
    model = load_YOLO_model(weight)
    results = detect_objects(model, output1)
    segmented, _ = segment_and_color_image(output1, results, face_area)

    # Accumulate the segmented result
    combined_mask = cv2.add(combined_mask, segmented)

# Multiply the combined_mask by prominence_factor if needed
prominence_factor = 2  # You can adjust this value
combined_mask = cv2.multiply(combined_mask, np.where(combined_mask > 0, prominence_factor, 1).astype(c_image.dtype))

# Overlay the combined_mask on the original face image
output = cv2.addWeighted(c_image, 1, combined_mask, 1, 0)

# Decrease the brightness of the output image (adjust alpha to control brightness)
alpha = 0.7  # You can adjust this value
darkened_output = cv2.addWeighted(output, 1 - alpha, output, 0, 0)

output_image_path = 'final_output_imageeee.png'
cv2.imwrite(output_image_path, darkened_output)



please enter image path: cvb.jpg



0: 640x640 1 -, 390.8ms
Speed: 18.8ms preprocess, 390.8ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 640)


acne: 70.26540201879568%



0: 640x640 2 Acne Detect - v2 2023-02-09 10-16ams, 297.4ms
Speed: 5.1ms preprocess, 297.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


darkcircles: 51.92393802935376%



0: 640x640 1 -, 288.7ms
Speed: 8.3ms preprocess, 288.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 800x800 (no detections), 336.9ms
Speed: 10.3ms preprocess, 336.9ms inference, 0.0ms postprocess per image at shape (1, 3, 800, 800)

0: 640x640 (no detections), 308.8ms
Speed: 3.6ms preprocess, 308.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 800x800 (no detections), 342.6ms
Speed: 17.2ms preprocess, 342.6ms inference, 0.0ms postprocess per image at shape (1, 3, 800, 800)

0: 640x640 (no detections), 313.8ms
Speed: 5.1ms preprocess, 313.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)


True

In [22]:
cv2.imshow("Input Image", combined_output)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [20]:
combined_output = c_image.copy()

In [9]:
from ultralytics import YOLO
import cv2
import numpy as np

model_paths = [
    'dark spots final.pt'
]

image_path = 'WhatsApp Image 2023-11-04 at 16.34.00.jpeg'

img = cv2.imread(image_path)
H, W, _ = img.shape

# Create an empty mask to accumulate all segmentations
combined_mask = np.zeros((H, W), dtype=np.uint8)

for model_path in model_paths:
    model = YOLO(model_path)

    results = model(img)

    for result in results:
        if result.masks is not None:  # Check if masks exist in the result
            for j, mask in enumerate(result.masks.data):
                mask = (mask.numpy() * 255).astype(np.uint8)  # Convert to np.uint8
                mask = cv2.resize(mask, (W, H))

                # Accumulate masks directly on combined_mask
                combined_mask = cv2.bitwise_or(combined_mask, mask)

# Find contours in the combined mask
contours, _ = cv2.findContours(combined_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

# Draw a boundary outline around each face contour
for contour in contours:
    cv2.drawContours(img, [contour], 0, (0, 255, 0), 2)

# Overlay the combined mask on the original image
segmented_img = cv2.bitwise_and(img, img, mask=combined_mask)

# Adjust transparency level
alpha = 0.7  # Adjust as needed
segmented_img = cv2.addWeighted(img, 1 - alpha, segmented_img, alpha, 0)

# Increase the brightness in the segmented area (make it brighter)
brightness_factor = 1.2  # Adjust the factor to control brightness
segmented_img = cv2.addWeighted(segmented_img, brightness_factor, np.zeros_like(segmented_img), 0, 0)

# Save or display the segmented image
cv2.imwrite('output_combined_3.png', segmented_img)


0: 512x640 1 -, 1 Hyperpigmentation, 336.1ms
Speed: 3.0ms preprocess, 336.1ms inference, 15.6ms postprocess per image at shape (1, 3, 512, 640)


True

In [1]:
def load_image():
    image = input('please enter image path: ')
    return cv2.imread(image)


KeyboardInterrupt

